##  파이썬과 MySQL 활용
1. 파이썬과 MySQL을 연결하는 모듈: mysqlclient, PyMySQL

        pip install mysqlclient
        pip install PyMySQL


In [1]:
import MySQLdb

In [1]:
import MySQLdb

In [ ]:
conn = MySQLdb.connect(host="192.168.0.3", user="newuser2", password="12345", database="testdb")
print(conn)

In [4]:
conn = MySQLdb.connect(host="192.168.0.10", user="newuser", password="1234", database="testdb")
print(conn)

<_mysql.connection open to '192.168.0.10' at 0000020435CCF218>


In [6]:
import pymysql
conn = pymysql.connect(host="192.168.0.10", user="newuser", password="1234", database="world")
print(conn)
conn.close()

In [35]:
config = {"host":"192.168.0.10", "user":"newuser", "password":"1234", "database":"world"}

In [71]:
conn = MySQLdb.connect(**config)
print(conn)

<_mysql.connection open to '192.168.0.10' at 0000020436BA0AB8>


In [72]:
# SELECT

sql = "select * from city limit 10"

cursor = conn.cursor()
cursor.execute(sql) #실행함수

# print(cursor)


#for data in cursor:
   # print(data)   # tuple로 묶어서 출력
    #print("%d %s %s %s %d"%data)


for a, b, c, d, e in cursor:
    print(a, b, c, d, e)
    
conn.close()

1 Kabul AFG Kabol 1780000
2 Qandahar AFG Qandahar 237500
3 Herat AFG Herat 186800
4 Mazar-e-Sharif AFG Balkh 127800
5 Amsterdam NLD Noord-Holland 731200
6 Rotterdam NLD Zuid-Holland 593321
7 Haag NLD Zuid-Holland 440900
8 Utrecht NLD Utrecht 234323
9 Eindhoven NLD Noord-Brabant 201843
10 Tilburg NLD Noord-Brabant 193238


In [34]:
# insert 작업

# table 만드는 작업
conn = MySQLdb.connect(**config)

sql = "create table if not exists tbltest(empno int, ename varchar(10), sal int, job varchar(10), mgr int, deptno int, hiredate datetime)"
cur = conn.cursor()
cur.execute(sql)

sql = "insert into tbltest(empno, ename, sal) values(1, 'scott', 4000)"
cur.execute(sql)
conn.commit()  # DML에서만 항사 commit 을 붙여야함


conn.close()

In [52]:
# 외부에서 입력받은 데이터를 insert

conn = MySQLdb.connect(**config)
cur = conn.cursor()

empno = 2
ename = 'miller'
sal = 1300

sql = "insert into tbltest(empno, ename, sal) values(" + str(empno) + ",'" + \
ename + "'," + str(sal) + ")"  # sal 문자열로 연결

#print(sql)

cur.execute(sql)
conn.commit()

conn.close()

In [56]:
# 간단히 insert 하는 방법
conn = MySQLdb.connect(**config)
cur = conn.cursor()

# 예외처리
try:
    empno = 3
    ename = 'tom'
    sal = 2000

    sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
    sql_data = (str(empno), str(ename), str(sal))

    cur.execute(sql, sql_data)
    conn.commit()

finally:
    conn.close()

In [62]:
# 데이터 여려개 입력하기

conn = MySQLdb.connect(**config)
cur = conn.cursor()


for i in range(10):
    sql = "insert into tbltest(empno, ename, sal) values(5, 'jane', 1000)"
    cur.execute(sql)
    conn.commit()
    
conn.close()


In [66]:
# cursor 객체의 fetchall() 모든데이터출력 , fetchmany() 다수의 데이터 , fetchone() 한개

conn = MySQLdb.connect(**config)
cur = conn.cursor()


try:
    cur.execute("select * from tbltest")
    
    #for i in cur.fetchall():
        #print(i)
        
    many = cur.fetchmany(4)
    for i in many:
        print(i)
        
        
    one = cur.fetchone()
    print(one)

finally:
    conn.close()

(1, 'scott', 4000, None, None, None, None)
(2, 'miller', 1300, None, None, None, None)
(3, 'tom', 2000, None, None, None, None)
(3, 'tom', 2000, None, None, None, None)
(3, 'tom', 2000, None, None, None, None)


In [69]:
# Dictcursor

conn = MySQLdb.connect(**config)
cur = conn.cursor(MySQLdb.cursors.DictCursor)


try:
    cur.execute("select * from tbltest")
    
    for i in cur.fetchall():
        #print(i[0], i[1], i[2]) 
        print(i["empno"], i["ename"], i["sal"])
        
finally:
    conn.close()

1 scott 4000
2 miller 1300
3 tom 2000
3 tom 2000
3 tom 2000
3 tom 2000
5 jane 1000
5 jane 1000
5 jane 1000
5 jane 1000
5 jane 1000
5 jane 1000
5 jane 1000
5 jane 1000
5 jane 1000
5 jane 1000


In [ ]:
# CSV파일을 DB에 저장

"""
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);
"""

In [88]:
# csv에는 더 많이씀
import csv
from datetime import datetime

file = csv.reader(open("input.csv"))
#print(file)

header = next(file) # CSV 모듈 자체가 ','' 로 구분해서 출력, 한 줄씩 불러오기, 
                    # 제목은 저장할 필요가 없기 때문에  그 다음 줄 부터 저장하기 위해 next로 출력함
#print(header)

#a = next(file)
#print(a)

conn = MySQLdb.connect(**config)
cur = conn.cursor()

try:
    cur.execute("delete from suppliers") # 12개만 저장
    conn.commit
    for row in file:   #next 는 자동적으로 입력됨
        #print(row)
        #cur.execute("insert into suppliers values(%s, %s, %s, %s, %s)", row)
        
        data = [] 
        for col in range(len(header)):   # 열의 개수 만큼 반복, 열 수정
            if col == 4:                 
                tmp = datetime.strptime(row[col], '%Y-%m-%d')
                data.append(tmp)
                
            else:
                data.append(row[col])
                
        cur.execute("insert into suppliers values(%s, %s, %s, %s, %s)", data)
        
    conn.commit()
    
finally:
    conn.close()

In [1]:
# DB에서 가져온 데이터를 파일로 저장

conn = MySQLdb.connect(**config)
cur = conn.cursor()

try:
    cur.execute("select * from suppliers where cost > 500000")
    
    
    file = csv.writer(open("output.csv", "w"), delimiter=",")  #csv 형식으로 저장
    
    header= ["Supplier Name","Invoice Number","Part Number","Cost","Purchase Date"]
    file.writerow(header)
    
    for row in cur:
        file.writerow(row)
    
finally:
    conn.close()

NameError: name 'MySQLdb' is not defined

## 파이썬과 SQLite

1. 로컬 DB로 사용할 수 있는 DBMS
    - ACCESS
    - SQLite
    - Microsoft SQL Sever Compact Edition
    - Firebird Embedded Server
    - H2
    - HSQL
    - Apache Derby
    ...
    
2. 특징
    - 파일 기반의 DBMS, 저메모리, 빠른 처리속도
    - 오픈 소스
    - 별도의 DB서버가 없어도 쉽고 편리하게 사용할 수 있는 Embeded SQL DB엔진
    - 안드로이드, 아이폰 등의 스마트폰에 기본 내장된 DB
    - 파이썬에도 내장
    - 표준 SQL 지원
    
    - 컬럼을 삭제하거나 변경하는 것 등이 제한
    - LEFT OUTHER JOIN만 지원
    - 읽기 전용 뷰만 제공 
    
3. 공식사이트
    http://sqlite.org
 
4. 클라이언트 툴
    http://www.sqliteexpert.com

In [1]:
import sqlite3

In [10]:
# CSV 파일을 읽어서 DB에 저장  # sqlite 는 DB가 없다고 해서 오류가 나는 것이 아니라 새로 DB를 만들기 때문에 확인이 필요
conn = sqlite3.connect("sqlitetest.db")
# print(conn)
cur= conn.cursor()

#tables = cur.execute("select * from sqlite_master where type='table'")
#for i in tables:
    #print(i)
import csv    
file = csv.reader(open("input.csv", "r"), delimiter =",") # csv 파일을 List 로 불러 온다
# print(file)

header = next(file)
# print(header)

for row in file:
    #print(row)
    cur.execute("insert into suppliers values(?, ?, ?, ?, ?)", row)   #MySQL은 %s로, data모듈의 차이
conn.commit()

cur.close()
conn.close()

In [11]:
# select로 확인

conn = sqlite3.connect("sqlitetest.db")
cur = conn.cursor()

cur.execute("select * from suppliers")

for row in cur.fetchmany(5):
    print(row)
    
cur.close()
conn.close()

('A', '001-1001', '2341', 500000.0, '2014-01-20')
('A', '001-1002', '2341', 500000.0, '2014-01-20')
('A', '001-1003', '5467', 750000.0, '2014-01-20')
('A', '001-1004', '5467', 750000.0, '2014-01-20')
('B', '001-1005', '7009', 250000.0, '2018-01-30')


In [26]:
# 메모리 DB
conn = sqlite3.connect(":memory:")
#print(conn)

cur = conn.cursor()
cur.execute("create table phonebook(name text, phonenumber text);")
cur.execute("insert into phonebook values(?,?);", ("tom", "010-111-1111"))
    
# 입력 데이터를 인자를 변수명으로 처리
sql = "insert into phonebook values(:vName, :vPhone)" # : 은 변수라고 지칭
cur.execute(sql, {"vName":"Johnson", "vPhone":"010-2222-2222"})

# 여러 개의 데이터를 한번에 입력
datalist = (('jane', '010-3333-3333'), ('jerry', '010-4444-4444'), ('marry','010-5555-5555'))
cur.executemany("insert into phonebook values(?, ?);", datalist)

# 작업한 내용을 script로 저장
f = open("dump.sql", "w")
for line in conn.iterdump():
    f.write("{}\n".format(line))
    
f.close()

cur.execute("select * from phonebook")
for row in cur:
    print(row)
    
conn.close()

('tom', '010-111-1111')
('Johnson', '010-2222-2222')
('jane', '010-3333-3333')
('jerry', '010-4444-4444')
('marry', '010-5555-5555')


In [28]:
# 스크립트 불러오기

conn = sqlite3.connect("sqlitetest.db")
cur = conn.cursor()

file = open("dump.sql", "r")
script = file.read()
#print(script)

cur.executescript(script)
conn.close()

## MongoDB 와 연동

    - pip install pymongo

##### 1. 서버연결

In [1]:
#import pymongo
from pymongo import MongoClient


In [7]:
#conn = pymongo.MongoClient("127.0.0.1", 27017)
conn = MongoClient("127.0.0.1", 27017)
print(conn)
conn.close()

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)


In [4]:
!pip install pymongo

##### 2. db연결

In [12]:
conn = MongoClient("mongodb://127.0.0.1:27017")
#dbname = conn.test2
#dbname = conn["test"]
dbname = conn.get_database("test")
print(dbname)
conn.close()

Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'test')


#### 3. collection 과 연결


In [3]:
conn = MongoClient("mongodb://127.0.0.1:27017")
dbname = conn.get_database("test")

collect = dbname.articles
print(collect)

conn.close()

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'articles')


#### 4. 데이터 입력

In [11]:
conn = MongoClient("mongodb://127.0.0.1:27017")
dbname = conn.get_database("test")

collect = dbname.people

#data = {"author":"Mike", "text":"my first blog post", "tag":["mongodb","python"]}
#collect.insert_one(data)

# 입력하면서 id 가져오기
data = {"author":"Tom", "text":"my first blog post", "tag":["mongodb","python"]}
post_id = collect.insert_one(data).inserted_id
print(post_id)

print(collect.count_documents({}))

for i in collect.find():
    print(i)
    
conn.close()    

7
{'_id': ObjectId('5fbded371f9c0a0c53b63548'), 'name': 'Abet', 'age': 20.0, 'score': 10.0}
{'_id': ObjectId('5fbded371f9c0a0c53b63549'), 'name': 'Betty', 'age': 20.0, 'score': 10.0}
{'_id': ObjectId('5fbded371f9c0a0c53b6354a'), 'name': 'Charlie', 'age': 23.0, 'skills': ['mongodb', 'nodejs']}
{'_id': ObjectId('5fbded371f9c0a0c53b6354b'), 'name': 'David', 'age': 23.0, 'score': 20.0}
{'_id': ObjectId('5fbdf4cf0e4248bed4c9e00e'), 'name': 'Elly', 'age': 17.0, 'score': 10.0}
{'_id': ObjectId('5fbf028c069510830f3dd8db'), 'author': 'Tom', 'text': 'my first blog post', 'tag': ['mongodb', 'python']}
{'_id': ObjectId('5fbf02e6069510830f3dd8dd'), 'author': 'Mike', 'text': 'my first blog post', 'tag': ['mongodb', 'python']}


#### 5. 데이터 입력하기(여러 개)

In [13]:
conn = MongoClient("mongodb://127.0.0.1:27017")
dbname = conn.get_database("test")
collect = dbname.people

print(collect.count_documents({}))

collect.insert_many([{"number":i} for i in range(10)])

print(collect.count_documents({}))

conn.close()

7
17


In [15]:
datas = []
for i in range(10):
    data = {"number": i}
    datas.append(data)

print(datas)

[{'number': 0}, {'number': 1}, {'number': 2}, {'number': 3}, {'number': 4}, {'number': 5}, {'number': 6}, {'number': 7}, {'number': 8}, {'number': 9}]


In [16]:
conn = MongoClient("mongodb://127.0.0.1:27017")
dbname = conn.get_database("test")
collect = dbname.people

data = list()
data.append({"name":"aron", "age":20})
data.append({"name":"tom", "age":27})
data.append({"name":"jerry", "age":10})
data.append({"name":"cardi", "age":24})

collect.insert_many(data)

conn.close()

##### 6. 조회

In [28]:
conn = MongoClient("mongodb://127.0.0.1:27017")
dbname = conn.get_database("test")
collect = dbname.people

conn.close()

# 전체 조회

#for i in collect.find():
    #print(i)
    
#for i in collect.find({"age":20}):
    #print(i)
# 나이가 20세 이상인 document 조회
#for i in collect.find({"age":{"$gte":23}}):
#    print(i)

#print(collect.find_one())

print(collect.find_one({"age":23}))
conn.close()

{'_id': ObjectId('5fbded371f9c0a0c53b6354a'), 'name': 'Charlie', 'age': 23.0, 'skills': ['mongodb', 'nodejs']}


#### 7. 수정

In [31]:
conn = MongoClient("mongodb://127.0.0.1:27017")
dbname = conn.get_database("test")
collect = dbname.people

collect.update_one({"author":"Tom"},{"$set":{"text":"my second blog post!"}})
print(collect.find_one({"author":"Tom"}))
conn.close()

{'_id': ObjectId('5fbf028c069510830f3dd8db'), 'author': 'Tom', 'text': 'my second blog post!', 'tag': ['mongodb', 'python']}
